# How to build Dashboards

One of BPTK's biggest strengths is the ability to easily connect it to interactive dashboards.

## Creating the model

We first need a BPTK model, let's create one.

First we do necessary imports and instantiate the model:

In [20]:
from BPTK_Py import Model
import BPTK_Py

model = Model(starttime=0.0, stoptime=10.0, dt=1.0, name="Test Model")

We have to define what the model does next.

In [21]:
stock = model.stock("balance")
taxes_accumulated = model.stock("tax_acc")
income = model.flow("income")
taxes = model.flow("taxes")
constantSalary = model.constant("salary")
constantTax = model.constant("tax")
constantTax.equation = 0.0
constantSalary.equation = 0.0

stock.equation = income - taxes
income.equation = constantSalary # income - tax
taxes.equation = constantSalary * constantTax

taxes_accumulated.equation = taxes

Lastly we have to register a scenario manager and scenario in BPTK.

In [22]:
scenario_manager = {"sm": {"model": model}}

bptk = BPTK_Py.bptk()
bptk.register_scenario_manager(scenario_manager)
bptk.register_scenarios(
    scenario_manager="sm",
    scenarios=
    {
        "testScenario":{
            "constants":
            {
                "salary":100.0,
                "tax":0.2
            }
        }
    }
)

Now we have our model setup. Next we want to display the output

## Display output
There are two ways to create dashboards in BPTK.

### The easy way (using SimpleDashboard class)
If you want to create dashboards the easy way, use the simple dashboard class. It automatically handles plot updates, connects widgets to the model and handles widget updating.

First, we have to import and instantiate the SimpleDashboard class.

In [23]:
from BPTK_Py.visualizations import SimpleDashboard
import ipywidgets as widgets

dashboard = SimpleDashboard(bptk, scenario_manager="sm", scenario="testScenario")

Now we have to create a few widgets. We want two sliders, one to update the tax rate and one to update the salary. When the sliders changed a graph plotting the balance is updated.

In [24]:
wdg_tax_slider = dashboard.get_float_slider(description="Tax rate", min=0.0, max=1.0, value=0.2, step=0.01, model_connection="tax")
wdg_salary_slider = dashboard.get_float_slider(description="Tax rate", min=0.0, max=1000.0, value=100.0, step=1.0, model_connection="salary")

plot = dashboard.add_plot(
    equations=["balance"], 
    names=["Balance"],
    title="Account Balance",
    x_label="Months",
    y_label="Balance",
)

controls = widgets.VBox([wdg_tax_slider, wdg_salary_slider])
display(plot)
display(controls)
dashboard.start()

Output()

It would be useful to see how much of salary gets paid as taxes. Luckily, adding multiple graphs is trivial.

In [25]:
plot2 = dashboard.add_plot(
    equations=["tax_acc"], 
    names=["Taxes"],
    title="Paid Taxes",
    x_label="Months",
    y_label="Taxes",
)

graph_tabs = widgets.Tab(children = [plot, plot2])
graph_tabs.set_title(0, 'Balance')
graph_tabs.set_title(1, 'Taxes')

display(graph_tabs)
display(controls)
dashboard.start()

### Advanced features

Bigger models tend to have more requirements than just updating a graph when a slider moves. In this example we create a more complex model to show advanced features.

First, let's update the model. This model has different tax rates depending on income. 

In [26]:
from BPTK_Py import sd_functions as sd

model = Model(starttime=0.0, stoptime=10.0, dt=1.0, name="Test Model")

stock = model.stock("balance")
taxes_accumulated = model.stock("tax_acc")
income = model.flow("income")
taxes = model.flow("taxes")
constantSalary = model.constant("salary")
constantSalary.equation = 0.0

stock.equation = income - taxes
income.equation = constantSalary # income - tax
taxes.equation = constantSalary * sd.lookup(constantSalary, "tax_rates")

taxes_accumulated.equation = taxes

scenario_manager = {"sm": {"model": model}}

bptk = BPTK_Py.bptk()
bptk.register_scenario_manager(scenario_manager)
bptk.register_scenarios(
    scenario_manager="sm",
    scenarios=
    {
        "testScenario2":{
            "base_points": {
                "tax_rates": [
                    [0.0, 0.1],
                    [250.0, 0.2],
                    [500.0, 0.25],
                    [750.0, 1.0],
                ]
            }
        }
    }
)

Next, we create the dashboard:

In [28]:
wdg_salary_slider = dashboard.get_float_slider(description="Salary", min=0.0, max=1000.0, value=100.0, step=1.0, model_connection="salary")

plot = dashboard.add_plot(
    equations=["balance"], 
    names=["Balance"],
    title="Account Balance",
    x_label="Months",
    y_label="Balance",
)

controls = widgets.VBox([wdg_salary_slider])
display(plot)
display(controls)
dashboard.start()

Output()